In [0]:
! kaggle

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 6, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [0]:
! cp kaggle.json /root/.kaggle/kaggle.json

In [0]:
! chmod 600 /root/.kaggle/kaggle.json

In [0]:
! kaggle datasets download thedagger/pokemon-generation-one

100% 1.08G/1.08G [00:09<00:00, 136MB/s]
100% 1.08G/1.08G [00:09<00:00, 118MB/s]


In [0]:
! unzip -qq pokemon-generation-one.zip

In [0]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D , Dropout
from keras.applications import ResNet50
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

In [0]:
base_model=ResNet50(weights='imagenet',include_top=False) 
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x)
x=Dense(1024,activation='relu')(x) 
x=Dense(512,activation='relu')(x) 
x = Dense(256 , activation='relu')(x)
preds=Dense(151,activation='softmax')(x) 

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [0]:
model=Model(inputs=base_model.input,outputs=preds)

In [0]:
for layer in model.layers[:175]:
    layer.trainable=False
for layer in model.layers[175:]:
    layer.trainable=True

In [0]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator=train_datagen.flow_from_directory('dataset/',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 subset = 'training',
                                                 shuffle=True)

Found 10657 images belonging to 151 classes.


In [0]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [0]:
step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=10)

Epoch 1/10
 92/333 [=======>......................] - ETA: 2:02 - loss: 4.7414 - acc: 0.0846

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


333/333 [==============================] - 149s 447ms/step - loss: 3.8930 - acc: 0.1811
Epoch 2/10
333/333 [==============================] - 142s 427ms/step - loss: 2.4021 - acc: 0.4017
Epoch 3/10
333/333 [==============================] - 142s 427ms/step - loss: 1.7785 - acc: 0.5304
Epoch 4/10
333/333 [==============================] - 143s 429ms/step - loss: 1.4116 - acc: 0.6128
Epoch 5/10
333/333 [==============================] - 142s 426ms/step - loss: 1.1719 - acc: 0.6724
Epoch 6/10
333/333 [==============================] - 142s 427ms/step - loss: 1.0010 - acc: 0.7104
Epoch 7/10
333/333 [==============================] - 142s 427ms/step - loss: 0.8673 - acc: 0.7461
Epoch 8/10
333/333 [==============================] - 142s 425ms/step - loss: 0.7530 - acc: 0.7813
Epoch 9/10
333/333 [==============================] - 142s 425ms/step - loss: 0.6563 - acc: 0.8016
Epoch 10/10
333/333 [==============================] - 142s 428ms/step - loss: 0.5911 - acc: 0.8232
